In [12]:
from nltk.stem import PorterStemmer
from nltk.tokenize import SpaceTokenizer
from nltk.corpus import stopwords
import re

tokenizer = SpaceTokenizer()
stemmer = PorterStemmer()

pipeline = [lambda s: re.sub('<[^>]*>', ' ', s),
            lambda s: re.sub("[^.'\w\s]", '', s),
            lambda s: re.sub('[\d]', '', s),
            lambda s: s.lower(),
            #lambda s: ' '.join(filter(lambda s: not (s in stopwords.words()), tokenizer.tokenize(s))),
            #lambda s: ' '.join(map(lambda t: stemmer.stem(t), tokenizer.tokenize(s)))
           ]

def preprocess_text(text, pipeline=pipeline):
    if len(pipeline) == 0:
        return text
    else:
        return preprocess_text(pipeline[0](text), pipeline[1:])

In [2]:
def preprocess_review(r):
    SUMMARY_INDEX = 8
    TEXT_INDEX = 9
    
    r[SUMMARY_INDEX] = preprocess_text(r[SUMMARY_INDEX])
    r[TEXT_INDEX] = preprocess_text(r[TEXT_INDEX])
    return r

In [7]:
import csv

def parallel_preprocess(input_file, output_file):
    # Load csv
    reviewsRDD = sc.textFile(input_file)
    reviewsRDD = reviewsRDD.mapPartitions(lambda x: csv.reader(x))

    # Save header
    header = reviewsRDD.first()

    # Extract header and preprocess
    reviewsRDD = reviewsRDD.filter(lambda x: x != header)\
                           .map(lambda x: preprocess_review(x))
    ppreviews = reviewsRDD.collect()

    # Insert header
    ppreviews.insert(0, header)
    ppreviews[0:2]

    # Save csv
    with open(output_file, 'wb') as f:
        writer = csv.writer(f)
        writer.writerows(ppreviews)

In [21]:
parallel_preprocess('../data/train.csv', '../data/sparktest.csv')

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 26.0 failed 1 times, most recent failure: Lost task 3.0 in stage 26.0 (TID 56, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/agustina/Documents/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/agustina/Documents/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/agustina/Documents/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
UnicodeEncodeError: 'ascii' codec can't encode character u'\xae' in position 415: ordinal not in range(128)

	at org.apache.spark.api.pyt